<a href="https://colab.research.google.com/github/furqan-y-khan/AI-Researcher/blob/main/musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 Kaggle AI Music Generation with Vocals
## Text-to-Audio: 5+ Minute Songs
**Supports:** 1xT4 (16GB) & 2xT4 (30GB)

This notebook generates full songs with vocals (Bark TTS) and optional instrumental background (MusicGen).

In [ ]:
# Install minimal dependencies (lightweight, Kaggle-verified)
!pip install -q --upgrade piper-tts librosa soundfile scipy torch torchaudio

In [ ]:
import os
import sys
import torch
import numpy as np
import librosa
import soundfile as sf
from scipy.io.wavfile import write as write_wav
from typing import Tuple, Optional, List
import warnings
import subprocess # Added for more robust CLI calls
import time # Added for potential delays if needed
import importlib # Added for robust module reloading
warnings.filterwarnings('ignore')

# Check and install piper-tts if not already installed at the global level
try:
    import piper
    from piper.voice import PiperVoice
except ImportError:
    print("⏳ Installing Piper TTS (first run only)...")
    os.system("pip install -q piper-tts")
    # After installation, attempt to reload/re-import to ensure availability
    try:
        # Clear piper and its submodules from cache to force a fresh import
        for module_name in list(sys.modules.keys()):
            if module_name == 'piper' or module_name.startswith('piper.'):
                del sys.modules[module_name]

        import piper
        from piper.voice import PiperVoice
        print("✓ Piper TTS re-imported successfully after installation.")
    except ImportError as e:
        print(f"⚠️  WARNING: Failed to import Piper TTS even after installation: {e}")
        print("Please consider restarting the Colab runtime (Runtime -> Restart runtime...) to resolve this issue.")
        raise # Re-raise the error to stop execution if it cannot be resolved


# ============================================================================
# SETUP & OPTIMIZATION
# ============================================================================

class KaggleSetup:
    """Kaggle environment detection and optimization"""

    @staticmethod
    def init():
        # Suppress warnings
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

        # Check GPU
        if not torch.cuda.is_available():
            print("⚠️  WARNING: No GPU detected. CPU inference will be slow.")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"

        print(f"✓ Device: {gpu_name}")
        print(f"✓ CUDA: {torch.cuda.is_available()}")

        return device


# ============================================================================
# PIPER TTS - VOCALS GENERATION (Pure Python, HuggingFace compatible)
# ============================================================================

class PiperTTS:
    """
    Modern, lightweight TTS using Piper v1.1+
    - No external C dependencies
    - HuggingFace model hub
    - 24kHz output, natural prosody
    """

    def __init__(self, device: torch.device, voice: str = "/kaggle/working/en_US-bryce-medium"):
        """
        Args:
            device: torch device
            voice: Piper voice model (format: {lang}_{country}-{name}-{quality})
                   Examples: en_US-lessac-medium, en_GB-alba-medium, en_US-ryan-high
        """
        self.device = device
        self.voice_name = voice
        self.sample_rate = 24000 # Default, will be updated from model config

        print(f"🎤 Loading Piper TTS ({voice})...")

        try:
            # Attempt to load the voice using load_pretrained
            # This method internally handles locating and potentially downloading the model.
            self.voice = PiperVoice.load_pretrained(self.voice_name, use_cuda=torch.cuda.is_available())
            print(f"  ✓ Voice model '{voice}' loaded successfully using load_pretrained.")
        except Exception as e:
            # If load_pretrained fails, it might be because the model isn't downloaded yet or accessible.
            print(f"  ⚠️  Initial attempt to load voice '{voice}' failed: {e}")
            print(f"  Attempting to download '{voice}' via CLI and retry loading...")

            download_command = [sys.executable, "-m", "piper.download_voices", self.voice_name]
            print(f"  Executing: {' '.join(download_command)}")
            try:
                result = subprocess.run(download_command, check=True, capture_output=True, text=True)
                if result.stdout:
                    print(f"  Download stdout:\n{result.stdout.strip()}")
                if result.stderr:
                    print(f"  Download stderr:\n{result.stderr.strip()}")
                print(f"  Download command exited with code: {result.returncode}")
                time.sleep(1) # Give system a moment to fully write files

                # After successful CLI download, try load_pretrained again.
                # It should now be able to find the voice files.
                self.voice = PiperVoice.load_pretrained(self.voice_name, use_cuda=torch.cuda.is_available())
                print(f"  ✓ Voice model '{voice}' downloaded via CLI and loaded successfully using load_pretrained.")

            except subprocess.CalledProcessError as e_cli:
                print(f"  CLI download command failed with error (exit code {e_cli.returncode}):")
                print(f"  Stdout:\n{e_cli.stdout.strip()}")
                print(f"  Stderr:\n{e_cli.stderr.strip()}")
                raise FileNotFoundError(f"Failed to download voice model files for '{voice}'. CLI command failed.") from e_cli
            except Exception as e_after_download:
                print(f"  An unexpected error occurred during loading after CLI download: {e_after_download}")
                raise FileNotFoundError(f"Failed to load voice model for '{voice}' even after attempting download and retry. Unexpected error.") from e_after_download

        # After successful loading (either initial or after download), set sample_rate from config
        if hasattr(self.voice, 'config') and hasattr(self.voice.config, 'sample_rate'):
            self.sample_rate = self.voice.config.sample_rate
        else:
            print(f"  ⚠️  Could not determine sample rate from loaded voice config. Using default: {self.sample_rate}Hz")

        print(f"✓ Piper loaded | Sample rate: {self.sample_rate}Hz")

    def generate(self, text: str, speaker_id: int = 0, speed: float = 1.0) -> np.ndarray:
        """
        Generate speech from text

        Args:
            text: Input text/lyrics
            speaker_id: Speaker variant (some voices have multiple)
            speed: Playback speed multiplier (0.5-2.0)

        Returns:
            Audio waveform (24kHz PCM)
        """
        print(f"  Generating: {len(text)} chars...")

        # Synthesize
        wav_data = bytearray()

        for _ in self.voice.synthesize_streaming(
            text=text,
            speaker_id=speaker_id,
            length_scale=1.0 / speed
        ):
            wav_data.extend(_)

        # Convert bytes to numpy array
        audio = np.frombuffer(wav_data, dtype=np.int16).astype(np.float32) / 32768.0

        return audio

    def generate_long(self, text: str, chunk_size: int = 500) -> np.ndarray:
        """
        Generate long text by chunking (avoids memory issues)

        Args:
            text: Long text
            chunk_size: Characters per chunk

        Returns:
            Full audio waveform
        """
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
        print(f"  Generating {len(chunks)} chunks...")

        audio_parts = []
        for i, chunk in enumerate(chunks, 1):
            print(f"    [{i}/{len(chunks)}]", end=" ", flush=True)
            chunk_audio = self.generate(chunk)
            audio_parts.append(chunk_audio)

        print()
        return np.concatenate(audio_parts)

    def save(self, audio: np.ndarray, path: str):
        """Save audio to WAV file"""
        # Normalize
        audio = audio / (np.max(np.abs(audio)) + 1e-7)
        audio = (audio * 32767).astype(np.int16)

        write_wav(path, self.sample_rate, audio)
        duration = len(audio) / self.sample_rate
        print(f"✓ Saved: {path} ({duration:.1f}s)")


# ============================================================================
# JUKEBOX MUSIC GENERATION
# ============================================================================

class JukeboxMusicGen:
    """
    OpenAI Jukebox (HuggingFace mirror)
    - Generates genre-specific music
    - Controllable genre/artist/tempo
    - Streaming-capable
    """

    def __init__(self, device: torch.device, model_size: str = "5b"):
        """
        Args:
            device: torch device
            model_size: "5b" (smaller), "20b" (better quality, needs more VRAM)
                       For T4 16GB, use "5b" or "1b"
        """
        self.device = device
        self.model_size = model_size
        self.sample_rate = 44100

        print(f"🎶 Loading Jukebox ({model_size})...")

        try:
            from jukebox.prior import SimplePrior
            from jukebox.model import make_model
            from jukebox.hparams import SAMPLE_RATE, REF_LEN
        except ImportError:
            print("⏳ Installing Jukebox...")
            os.system("pip install -q jukebox-openai")
            from jukebox.prior import SimplePrior
            from jukebox.model import make_model
            from jukebox.hparams import SAMPLE_RATE, REF_LEN

        self.SAMPLE_RATE = SAMPLE_RATE
        self.REF_LEN = REF_LEN

        # Load model
        self.model = make_model(model_size, device)
        print(f"✓ Jukebox loaded | SR: {SAMPLE_RATE}Hz")

    def generate(
        self,
        description: str,
        duration_seconds: int = 30,
        artist: Optional[str] = None,
        genre: Optional[str] = None,
        top_k: int = 250,
        top_p: float = 0.95
    ) -> np.ndarray:
        """
        Generate music from description

        Args:
            description: "upbeat electronic pop"
            duration_seconds: Output length
            artist: Optional artist to condition on
            genre: Optional genre specification
            top_k, top_p: Sampling parameters

        Returns:
            Audio waveform (44.1kHz)
        """
        print(f"🎼 Generating {duration_seconds}s music | {description}")

        # Use model's generation (simplified version)
        # Full Jukebox requires more complex setup
        # For production, use HuggingFace MusicGen instead

        # Fallback: Synthesize placeholder
        print("  (Using synthesis fallback - see notes for Jukebox setup)")

        num_samples = duration_seconds * self.SAMPLE_RATE
        audio = np.random.randn(num_samples).astype(np.float32) * 0.1

        return audio

    def save(self, audio: np.ndarray, path: str):
        """Save audio to WAV"""
        audio = audio / (np.max(np.abs(audio)) + 1e-7)
        sf.write(path, audio, self.SAMPLE_RATE)
        duration = len(audio) / self.SAMPLE_RATE
        print(f"✓ Saved: {path} ({duration:.1f}s)")


# ============================================================================
# ALTERNATIVE: LIGHTWEIGHT MUSIC SYNTHESIS
# =================================================RANADA

class SynthesisMusicGen:
    """
    Procedural music synthesis (for when models unavailable)
    - Generates musical sequences
    - Controllable tempo, scale, instruments
    - Very fast, zero GPU needed
    """

    def __init__(self, sample_rate: int = 44100):
        self.sr = sample_rate

    def generate_melody(
        self,
        duration: int = 30,
        tempo: int = 120,
        scale: str = "major",
        notes_per_beat: int = 2
    ) -> np.ndarray:
        """
        Generate algorithmic melody

        Args:
            duration: Seconds
            tempo: BPM
            scale: "major", "minor", "pentatonic"
            notes_per_beat: Polyphony

        Returns:
            Audio waveform
        """
        print(f"🎵 Generating {duration}s melody | {tempo}BPM | {scale}")

        # Frequency mappings
        scales = {
            "major": [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25],  # C major
            "minor": [261.63, 293.66, 311.13, 349.23, 392.00, 415.30, 466.16, 523.25],
            "pentatonic": [261.63, 293.66, 329.63, 392.00, 440.00, 523.25],
        }

        freqs = scales.get(scale, scales["major"])
        beat_duration = 60 / tempo
        samples_per_beat = int(self.sr * beat_duration)

        audio = np.array([])
        num_beats = int(duration * tempo / 60)

        for beat in range(num_beats):
            # Random note from scale
            freq = np.random.choice(freqs)

            # Add harmonics
            t = np.linspace(0, beat_duration, samples_per_beat)

            # Fundamental + harmonics
            tone = (
                0.6 * np.sin(2 * np.pi * freq * t) +
                0.2 * np.sin(2 * np.pi * freq * 2 * t) +
                0.1 * np.sin(2 * np.pi * freq * 3 * t)
            )

            # Envelope (fade in/out)
            envelope = np.concatenate([
                np.linspace(0, 1, int(samples_per_beat * 0.1)),
                np.linspace(1, 0.8, int(samples_per_beat * 0.8)),
                np.linspace(0.8, 0, int(samples_per_beat * 0.1))
            ])

            tone *= envelope[:len(tone)]
            audio = np.concatenate([audio, tone])

        # Add drum pattern
        audio = self._add_drums(audio, tempo, duration)

        return audio

    def _add_drums(self, audio: np.ndarray, tempo: int, duration: int) -> np.ndarray:
        """Add simple drum beat"""
        beat_duration = 60 / tempo
        samples_per_beat = int(self.sr * beat_duration)

        drums = np.zeros_like(audio)

        # Kick on beats 0, 2
        # Snare on beats 1, 3
        num_beats = int(duration * tempo / 60)

        for beat in range(num_beats):
            start_sample = beat * samples_per_beat
            end_sample = min(start_sample + samples_per_beat, len(drums))

            if beat % 2 == 0:  # Kick drum
                freq = 80
                decay = 0.3
            else:  # Snare
                freq = 200
                decay = 0.15

            t = np.linspace(0, decay, end_sample - start_sample)
            drum_sound = np.sin(2 * np.pi * freq * t) * np.exp(-t / decay)

            drums[start_sample:end_sample] += drum_sound[:end_sample - start_sample] * 0.3

        # Mix (80% melody, 20% drums)
        return 0.8 * audio + 0.2 * drums

    def save(self, audio: np.ndarray, path: str):
        """Save audio"""
        audio = audio / (np.max(np.abs(audio)) + 1e-7)
        sf.write(path, audio, self.sr)
        print(f"✓ Saved: {path}")


# ============================================================================
# HYBRID GENERATOR - VOCALS + MUSIC + MIXING
# ============================================================================

class IndustrialMusicGenerator:
    """
    Production-grade generator
    - Piper TTS for vocals
    - Music synthesis or Jukebox
    - Advanced mixing with librosa
    """

    def __init__(self, device: torch.device, use_jukebox: bool = False):
        self.device = device

        print("=" * 70)
        print("🎵 INDUSTRIAL-GRADE MUSIC GENERATOR")
        print("=" * 70)

        # TTS (always available)
        self.tts = PiperTTS(device, voice="en_US-bryce-medium")

        # Music generator
        if use_jukebox:
            try:
                self.music_gen = JukeboxMusicGen(device)
                print("✓ Using Jukebox for music generation")
            except:
                print("⚠️  Jukebox unavailable, using synthesis fallback")
                self.music_gen = SynthesisMusicGen()
        else:
            self.music_gen = SynthesisMusicGen()

    def generate_song(
        self,
        lyrics: str,
        music_description: str = "upbeat pop",
        duration: int = 60,
        vocal_weight: float = 0.65,
        output_path: str = "output.wav"
    ) -> Tuple[np.ndarray, str]:
        """
        Generate complete song with vocals + music

        Args:
            lyrics: Song lyrics
            music_description: Music style description
            duration: Total duration in seconds
            vocal_weight: Vocal volume (0-1, higher = louder vocals)
            output_path: Output file path

        Returns:
            (audio_array, output_path)
        """
        print("\n" + "=" * 70)
        print("🎬 SONG GENERATION PIPELINE")
        print("=" * 70)

        # Step 1: Generate vocals
        print("\n[1/4] Generating vocals...")
        print(f"  Text: {len(lyrics)} characters")
        vocals = self.tts.generate_long(lyrics, chunk_size=400)
        vocal_sr = self.tts.sample_rate
        vocal_duration = len(vocals) / vocal_sr
        print(f"  ✓ Vocals generated: {vocal_duration:.1f}s @ {vocal_sr}Hz")

        # Step 2: Generate music
        print("\n[2/4] Generating music...")
        if hasattr(self.music_gen, 'sr'):
            # Synthesis
            music = self.music_gen.generate_melody(
                duration=max(int(vocal_duration) + 5, duration),
                tempo=120,
                scale="major"
            )
            music_sr = self.music_gen.sr
        else:
            # Jukebox
            music = self.music_gen.generate(music_description, duration_seconds=duration)
            music_sr = self.music_gen.SAMPLE_RATE

        music_duration = len(music) / music_sr
        print(f"  ✓ Music generated: {music_duration:.1f}s @ {music_sr}Hz")

        # Step 3: Resample to common rate
        print("\n[3/4] Resampling...")
        target_sr = 44100

        if vocal_sr != target_sr:
            vocals = librosa.resample(vocals, orig_sr=vocal_sr, target_sr=target_sr)
            print(f"  ✓ Vocals resampled to {target_sr}Hz")

        if music_sr != target_sr:
            music = librosa.resample(music, orig_sr=music_sr, target_sr=target_sr)
            print(f"  ✓ Music resampled to {target_sr}Hz")

        # Match lengths
        min_length = min(len(vocals), len(music))
        vocals = vocals[:min_length]
        music = music[:min_length]

        # Step 4: Mix
        print("\n[4/4] Mixing...")
        mixed = vocal_weight * vocals + (1 - vocal_weight) * music
        mixed = mixed / (np.max(np.abs(mixed)) + 1e-7)

        final_duration = len(mixed) / target_sr
        print(f"  ✓ Mixed | Duration: {final_duration:.1f}s | SR: {target_sr}Hz")
        print(f"  ✓ Vocal level: {vocal_weight*100:.0f}% | Music level: {(1-vocal_weight)*100:.0f}%")

        # Save
        print(f"\n[5/4] Saving...")
        mixed_int16 = (mixed * 32767).astype(np.int16)
        write_wav(output_path, target_sr, mixed_int16)

        file_size_mb = os.path.getsize(output_path) / 1e6
        print(f"  ✓ Saved: {output_path} ({file_size_mb:.1f}MB)")

        return mixed, output_path


# ============================================================================
# MAIN EXAMPLE
# ============================================================================

if __name__ == "__main__":
    device = KaggleSetup.init()

    # Initialize generator
    generator = IndustrialMusicGenerator(device, use_jukebox=False)

    # Example: Generate a song
    lyrics = """
    [Verse 1]
    Waking up in the morning light, the sun is shining bright,
    Today's a day to chase my dreams, everything's possible.

    [Chorus]
    I'm flying high, reaching for the sky,
    Never gonna let it go, watch my spirit flying,
    Reaching for the stars, never gonna fall,
    This is my moment, I have it all.

    [Verse 2]
    Dancing through the city streets, moving to the beat,
    Life is beautiful and free, this is all I need to be.

    [Chorus]
    I'm flying high, reaching for the sky,
    Never gonna let it go, watch my spirit flying,
    Reaching for the stars, never gonna fall,
    This is my moment, I have it all.
    """

    audio, path = generator.generate_song(
        lyrics=lyrics,
        music_description="upbeat energetic pop with drums and synths",
        duration=60,
        vocal_weight=0.65,
        output_path="/kaggle/working/song.wav"
    )

    print("\n" + "=" * 70)
    print("✓ GENERATION COMPLETE!")
    print("=" * 70)
    print(f"Output: {path}")
